In [365]:

# import pandas as pd

# def split_into_batches(df, batch_size=1000):
#     """Yield consecutive batches of given size from DataFrame."""
#     for i in range(0, len(df), batch_size):
#         yield df.iloc[i:i + batch_size]

# # Load the Excel file and sheet
# file_path = "main_data/human_chatgpt_filtered_data.csv"  # Change this to your file
# df = pd.read_csv(file_path)

# # Filter rows where chatgpt_discard == 0
# filtered_df = df[df["chatgpt_discard"] == 0]

# print(f"Original DataFrame: {df.shape}")
# print(f"Filtered DataFrame: {filtered_df.shape}")

# ## shuffle the data
# filtered_df = filtered_df.sample(frac=1).reset_index(drop=True)

# ## remove the chatgpt_discard column
# filtered_df = filtered_df.drop(columns=['chatgpt_discard'])
# # rename column point to review_point
# filtered_df = filtered_df.rename(columns={"point": "review_point"})

# # Split into batches of 1000
# batches = list(split_into_batches(filtered_df, batch_size=1000))


In [366]:
# import os
# import pandas as pd
# import re

# def clean_cell(value):
#     """Remove illegal characters from cell values."""
#     if isinstance(value, str):
#         return re.sub(r'[\x00-\x1F\x7F]', '', value)  # Remove control characters
#     return value  # Return non-string values unchanged

# for i, batch in enumerate(batches):
#         file_path = f"main_data/batches/batch_{i}.xlsx"
#         print(f"Writing {file_path}...")
#         df = batch.copy()
#         # Clean the dataframe to remove illegal characters
#         df = df.applymap(clean_cell)

#         with pd.ExcelWriter(file_path, engine='openpyxl') as writer:
#             for aspect in ['actionability', 'grounding_specificity', 'verifiability', 'helpfulness']:
#                 df.to_excel(writer, sheet_name=aspect, index=False)

In [367]:
import pandas as pd
from openai import OpenAI, AsyncOpenAI
import os
import dotenv
dotenv.load_dotenv()
from prompt import *
client = OpenAI(api_key=os.environ.get("review_evaluation_mbzuai"))

model_name = 'gpt-4o'

import json 
import pandas as pd
aspects = [ 'actionability', 'grounding_specificity','verifiability_extraction', 'helpfulness']
all_incontext_examples = pd.read_excel('/home/abdelrahman.sadallah/mbzuai/review_rewrite/chatgpt/test_data/in_context_examples.xlsx', sheet_name=None)



BATCH = "random"
data_path = f'main_data/batches/batch_{BATCH}.xlsx'
TEMP = 0.1
num_examples_per_label = 5



In [368]:
import random
def get_prompt(review_point,aspect, in_context_examples,num_examples_per_label=1,):

    prompt = ''
    examples = ''
    examples_str = []
    ##3 group examples by the label and choose a random example from each group
    for label in in_context_examples[f'{aspect}_label'].unique():
        for _ in range(num_examples_per_label):
            ## keep sampling a line till it is not the same as the currrent review point
            while True:
                row = in_context_examples[in_context_examples[f'{aspect}_label']==label].sample(1)
                row = row.iloc[0]
                if row['review_point'] != review_point:
                    break
            
            score = row[f'{aspect}_label']
            rationale = row['rationale']


            examples_str.append(f'''
Review Point: {row['review_point']}
rationale: {rationale}
score: {score}
''')
    ## shuffle the list 
    random.shuffle(examples_str)
    examples = '\n'.join(examples_str)
    
    ## for verifiability, we have two tasks
    prompt = BASE_PROMPT_EXAMPLES.format(review_point=review_point,aspect=aspect,aspect_description=ASPECTS[aspect],examples=examples)
    return prompt

In [369]:
def chatgpt_batch_inf(test_data,aspect, batch,temp=0, num_examples_per_label=1):
   in_context_examples =  all_incontext_examples[aspect]
   lines = []

   for i,row in test_data.iterrows():
      review_point = row['review_point']
      prompt = get_prompt(review_point=review_point,aspect=aspect, in_context_examples=in_context_examples, num_examples_per_label=num_examples_per_label)   
      line = {
         "custom_id": f"{row['id']}", 
         "method": "POST", 
         "url": "/v1/chat/completions", 
         "body": {"model": model_name,
         "temperature": temp,
         "messages": 
         [{"role": "system", "content": SYSTEM_PROMPT},
         {"role": "user", "content": prompt}],}}
      lines.append(line)

   print(f'sample of the prompts is {lines[0]}')

   ### Write batch input file
   batch_file_path = f"batch_data/main_data_batch_{batch}_{aspect}_input.jsonl"
   with open(batch_file_path, 'w') as f:
      for l in lines:
         json.dump(l, f)
         f.write('\n')

   ### upload the batch file
   batch_input_file = client.files.create(
   file=open(batch_file_path, "rb"),
   purpose="batch")

   ### create the batch request
   batch_input_file_id = batch_input_file.id
   batch_data = client.batches.create(
      input_file_id=batch_input_file_id,
      endpoint="/v1/chat/completions",
      completion_window="24h",
      metadata={
         "description": f"batch file for  {aspect} model gpt-4o, temperature {temp} for batch {batch}",
      })
   batch_metadata = {
      "batch_id": batch_data.id,
      "aspect": aspect,
      "batch_input_file_id": batch_input_file_id,
      "batch_file_path": batch_file_path,
      "batch": batch,
   }

   with open(f"batch_data/main_data_batch_{batch}_{aspect}_input_meta_data.json", 'w') as f:
      json.dump(batch_metadata, f, indent=4)
      
   print(f"Batch file for {aspect}, and  batch {batch} is created and uploaded to the server")


In [370]:
# for aspect in aspects:
    # if aspect in ['verifiability_extraction']:
    #     sheet_name = 'verifiability'
    # else:
    #     sheet_name = aspect
    # test_data = pd.read_excel(data_path, sheet_name=sheet_name)
    # ### check if test_data has id column and add one if not
    # if 'id' not in test_data.columns:
    #     test_data['id'] = range(1, len(test_data) + 1)
    #     with pd.ExcelWriter(data_path, mode='a', engine='openpyxl', if_sheet_exists='replace') as writer:
    #         test_data.to_excel(writer, sheet_name=sheet_name, index=False)
        
#     chatgpt_batch_inf(test_data=test_data,
#                         aspect=aspect, 
#                         batch=BATCH,
#                         temp=TEMP,
#                         num_examples_per_label=num_examples_per_label)



############################# VERIFIABILITY CODE ########################################
# check for data of verifiability_extraction if it's done, then do the verifiability_verification

# extraction_results_path = f'outputs/main_data_batch_{BATCH}_results.xlsx'
# for file in os.listdir('batch_data'):
#     if 'verifiability_extraction' in file and 'meta_data' in file :
#         verifiability_extraction_batch_data = json.load(open(f'batch_data/{file}'))
# if client.batches.retrieve(verifiability_extraction_batch_data['batch_id']).status == 'completed':
#     ## get the saved data, and only consider the ones with labes yes
#     test_data = pd.read_excel(extraction_results_path, sheet_name='verifiability_extraction')
#     test_data = test_data[test_data['chatgpt_verifiability_extraction_score']!='X']

#     print(f'Verifiability extraction is done, moving to verifiability verification')
#     print(f'Size of verifiability verification data is {test_data.shape[0]}')
#     chatgpt_batch_inf(test_data=test_data,aspect='verifiability_verification', 
#                       batch=BATCH, 
#                       temp=TEMP,
#                       num_examples_per_label=num_examples_per_label)

In [371]:
import re

def extract_rationale_and_score(text):
    valid_score_values = ["yes", "no",'X', '1', '2', '3', '4', '5']
    # Extract rationale
    rationale_match = re.search(r"(?i)\*?\*?rationale:\*?\*?\s*(.*?)(?=\n\s*\*?\*?(?i)score:\*?\*?|$)", text, re.DOTALL)
    rationale = rationale_match.group(1).strip() if rationale_match else None
    
    # Extract score (original pattern)
    score_match = re.search(r"(?i)\*?\*?score:\*?\*?\s*(\d+|yes|no)[\s.]*$", text)
    score = score_match.group(1).lower() if score_match else None
    
    if not score:
        # Try to extract a number within five words after "score" or "rated"
        score_context_match = re.search(r'(?i)(score|rated)[:\s]+((?:\S+\s+){0,5}?)["\']?(\d+)', text)
        if score_context_match:
            score = score_context_match.group(3)  # Extract the number if found

    if score:
        ## remove trailing spaces and dots from the score
        score = score.strip().rstrip('.')
        score = str(score)
        # Ensure final score is valid
        if score not in valid_score_values:
            score = None  # Invalidate if it's not a valid score
    
    return {"rationale": rationale, "score": score}

In [372]:
test = '''rationale: The review point suggests investigating the use of contextualized vectors for note embeddings by proposing three additional ablation studies. However, it does not provide any justification or reasoning for why these specific ablations are necessary or how they would contribute to the understanding of the issue. The lack of explanation or supporting evidence makes it difficult for the authors to grasp the importance or relevance of the suggested ablations. Consequently, the comment is rated as "1: Unverifiable" due to the absence of verifiable support or rationale.'''
extract_rationale_and_score(test)

{'rationale': 'The review point suggests investigating the use of contextualized vectors for note embeddings by proposing three additional ablation studies. However, it does not provide any justification or reasoning for why these specific ablations are necessary or how they would contribute to the understanding of the issue. The lack of explanation or supporting evidence makes it difficult for the authors to grasp the importance or relevance of the suggested ablations. Consequently, the comment is rated as "1: Unverifiable" due to the absence of verifiable support or rationale.',
 'score': '1'}

In [373]:
def retrive_batch_and_save_results(batch_data, chatgpt_output_path):
    aspect = batch_data['aspect']
    batch_id = batch_data['batch_id']
    output_file_id = client.batches.retrieve(batch_id).output_file_id
    chatgpt_response =  client.files.content(output_file_id)
    file_path = chatgpt_output_path
    with open(file_path, 'w') as file:
        file.write(chatgpt_response.text + '\n')

In [374]:
import re
import pandas as pd

aspects = [ 'actionability', 'grounding_specificity', 'verifiability', 'helpfulness']
def write_results_in_original_file(batch_data, chatgpt_output_path, raw_data_path, chatgpt_input_path, write_path):
    errors = 0
    aspect = batch_data['aspect']

    if aspect in ['verifiability_extraction']:
        sheet_name = 'verifiability'
    elif aspect in ['verifiability_verification']:
        sheet_name = 'verifiability_extraction'
    else:
        sheet_name = aspect
    
    chatgpt_response = pd.read_json(chatgpt_output_path, lines=True)
    raw_data_df = pd.read_excel(raw_data_path, sheet_name=sheet_name)
    chatgpt_input = pd.read_json(chatgpt_input_path, lines=True)

    ### iterate over the review_points in the raw dataframe and make sure they are aligned with the chatgpt input data
    final_df = []
    for i in range(raw_data_df.shape[0]):
        id = raw_data_df.iloc[i]['id']

        row = raw_data_df.iloc[i].copy()

        ## if this is verifiability_verification, and we don't find the id, then the was an X case, and we don't do anything
        if  aspect == 'verifiability_verification' and  id not in chatgpt_response['custom_id'].values:
            continue

        chatgpt_row = chatgpt_response[chatgpt_response['custom_id']==id]

        input = chatgpt_input[chatgpt_input['custom_id']==id].iloc[0]['body']['messages'][1]['content']

        chatgpt_row = chatgpt_row.iloc[0].copy()

        answer = chatgpt_row['response']['body']['choices'][0]['message']['content']

        extracted_output = extract_rationale_and_score(answer)
        rationale, score = extracted_output['rationale'], extracted_output['score']
        aspect_save_name = aspect
        if aspect in ['verifiability_extraction', 'verifiability_verification']:
            score = 'X' if score == 'no' else score
        if aspect in ['verifiability_verification']:
            aspect_save_name = 'verifiability'
        row [f'chatgpt_{aspect_save_name}_score'] = score
        row [f'chatgpt_{aspect_save_name}_rationale'] = rationale
        row [f'prompt'] = input

        if not score:
            errors += 1
            print(f"can't process {answer}")
            continue


        final_df.append(row)

    final_df = pd.DataFrame(final_df)

    ## if the aspect is verifiability_verification, we need to merge the results with the extraction results
    ## rename the aspect to verifiability
    if aspect == 'verifiability_verification':
        extraction_results = raw_data_df[raw_data_df['chatgpt_verifiability_extraction_score']=='X']
        extraction_results['chatgpt_verifiability_score'] = extraction_results['chatgpt_verifiability_extraction_score']
        extraction_results['chatgpt_verifiability_rationale'] = extraction_results['chatgpt_verifiability_extraction_rationale']
        extraction_results.drop(['chatgpt_verifiability_extraction_score','chatgpt_verifiability_extraction_rationale'], axis=1, inplace=True)
        final_df = pd.concat([final_df, extraction_results], axis=0)

        


    if aspect in [ 'verifiability_verification']:
        aspect = 'verifiability'
    ## remove all the columns that related to another aspects
    # for any column if it has the aspect name of other aspects, remove it
    ## grap the other aspects list
    ## remove all cloulmns expptr for this hlise 
    exclude = ['review_point','paper_id','id','venue','focused_review','batch','prompt']
    for col in final_df.columns:
        if col in exclude:
            continue
        if aspect in col:
            continue
        final_df.drop(col, axis=1, inplace=True)         
    ## rewrite the existing sheet, but keep the other sheets
    file_path = write_path

    ## if the file does not exist, create a new one
    if not os.path.exists(file_path):
        final_df.to_excel(file_path, sheet_name=aspect, index=False)
    else:
        with pd.ExcelWriter(file_path, mode='a', engine='openpyxl', if_sheet_exists='replace') as writer:
            final_df.to_excel(writer, sheet_name=aspect, index=False)
    
    print(final_df[f'chatgpt_{aspect_save_name}_score'].value_counts())

    print(f'number of errors for {aspect} is {errors}. The number of rows processed is {final_df.shape[0]}')

In [375]:
import os
import json
for file in os.listdir('batch_data'):
    if 'input_meta_data.json' in file and str(BATCH) in file:
        batch_data = json.load(open(f'batch_data/{file}'))

        if client.batches.retrieve(batch_data['batch_id']).status == 'completed':
            print(f"Batch file for {batch_data['aspect']} and batch {batch_data['batch']} has been completed")
            aspect = batch_data['aspect']

            chatgpt_output_path = f"batch_output/main_data_batch_{BATCH}_{aspect}_chatgpt_output.jsonl"

            raw_data_path = f"main_data/batches/batch_{BATCH}.xlsx"

            chatgpt_input_path =  f"batch_data/main_data_batch_{BATCH}_{aspect}_input.jsonl"
            write_path = f"outputs/main_data_batch_{BATCH}_results.xlsx"

            retrive_batch_and_save_results (batch_data, chatgpt_output_path=chatgpt_output_path)

            if aspect == 'verifiability_verification':
                raw_data_path = write_path
                
            write_results_in_original_file(batch_data,
                                        chatgpt_output_path=chatgpt_output_path,
                                        raw_data_path=raw_data_path,
                                        chatgpt_input_path=chatgpt_input_path,
                                        write_path=write_path)
            
        else:
            print(f"Batch file for {batch_data['aspect']} and batch {batch_data['batch']} has not been completed yet")



Batch file for grounding_specificity and batch random has been completed
chatgpt_grounding_specificity_score
3    88
5    65
1    24
2    15
4     8
Name: count, dtype: int64
number of errors for grounding_specificity is 0. The number of rows processed is 200
Batch file for helpfulness and batch random has been completed
chatgpt_helpfulness_score
3    97
4    71
2    19
5    10
1     3
Name: count, dtype: int64
number of errors for helpfulness is 0. The number of rows processed is 200
Batch file for verifiability_verification and batch random has been completed
can't process rationale: The review point presents a claim regarding the difficulty of implementing and testing predictions with simulations in the context of recognition lists and old vs. new judgments. The reviewer acknowledges that the argument makes sense in principle but raises concerns about practical implementation. However, the comment lacks specific examples, references, or detailed reasoning to support the claim about 

/tmp/ipykernel_3689161/329380370.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  extraction_results['chatgpt_verifiability_score'] = extraction_results['chatgpt_verifiability_extraction_score']
/tmp/ipykernel_3689161/329380370.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  extraction_results['chatgpt_verifiability_rationale'] = extraction_results['chatgpt_verifiability_extraction_rationale']
/tmp/ipykernel_3689161/329380370.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of 

chatgpt_verifiability_score
3    92
1    40
2    24
X    24
4    13
5     6
Name: count, dtype: int64
number of errors for verifiability is 1. The number of rows processed is 199
Batch file for verifiability_extraction and batch random has been completed
chatgpt_verifiability_extraction_score
yes    176
X       24
Name: count, dtype: int64
number of errors for verifiability_extraction is 0. The number of rows processed is 200
Batch file for actionability and batch random has been completed
chatgpt_actionability_score
3    72
5    41
1    40
2    27
4    20
Name: count, dtype: int64
number of errors for actionability is 0. The number of rows processed is 200
